In [ ]:

from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Step 1: Load embedding model and LLM
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
llm = ChatOllama(model="gemma3:4b")

# Step 2: Create sample context chunks
chunks = [
    "The Eiffel Tower is located in Paris.",
    "Python is a programming language used for data science.",
    "OpenAI develops large language models like ChatGPT.",
    "The Great Wall of China is visible from space.",
    "Water boils at 100 degrees Celsius under standard conditions.",
    "The mitochondrion is the powerhouse of the cell.",
    "Mount Everest is the tallest mountain above sea level.",
    "Shakespeare wrote Hamlet and Macbeth.",
    "Tesla produces electric vehicles and solar products.",
    "Amazon is a large online retailer founded by Jeff Bezos.",
    "The Moon orbits the Earth approximately every 27.3 days.",
    "The speed of light is about 299,792 kilometers per second.",
    "Albert Einstein developed the theory of relativity.",
    "Venus is the second planet from the Sun.",
    "The Pacific Ocean is the largest and deepest ocean on Earth.",
    "Bacteria are single-celled microorganisms.",
    "The capital of Japan is Tokyo.",
    "Photosynthesis occurs in the chloroplasts of plant cells.",
    "Leonardo da Vinci painted the Mona Lisa.",
    "Bitcoin is a decentralized digital currency."
]

# Step 3: Embed chunks and index them
chunk_vectors = embed_model.encode(chunks)
index = faiss.IndexFlatL2(chunk_vectors.shape[1])
index.add(np.array(chunk_vectors))

# Step 4: Define prompt template
template = """Use the following context to answer the question:
Context: {context}
Question: {question}
Answer:"""
prompt = PromptTemplate.from_template(template)
chain = LLMChain(llm=llm, prompt=prompt)

# Step 5: Define questions and run chain with retrieved context
questions = [
    "Where is the Eiffel Tower?",
    "Who painted the Mona Lisa?",
    "What does Tesla produce?",
    "What is the speed of light?",
    "Where does photosynthesis happen in plant cells?"
]



In [ ]:
# Run the chain and print results nicely
results = []
for question in questions:
    query_vec = embed_model.encode([question])
    _, idx = index.search(np.array(query_vec), k=1)
    top_chunk = chunks[idx[0][0]]
    answer = chain.run(context=top_chunk, question=question).strip()
    results.append((question, top_chunk, answer))

# Nicely formatted output
for i, (question, context, answer) in enumerate(results, 1):
    print(f"🧠 Question {i}: {question}")
    print(f"📚 Retrieved Context: {context}")
    print(f"✅ Answer: {answer}")
    print("-" * 70)


In [ ]:
🧠 Question 1: Where is the Eiffel Tower?
📚 Retrieved Context: The Eiffel Tower is located in Paris.
✅ Answer: Paris.
----------------------------------------------------------------------
🧠 Question 2: Who painted the Mona Lisa?
📚 Retrieved Context: Leonardo da Vinci painted the Mona Lisa.
✅ Answer: Leonardo da Vinci.
----------------------------------------------------------------------
🧠 Question 3: What does Tesla produce?
📚 Retrieved Context: Tesla produces electric vehicles and solar products.
✅ Answer: Tesla produces electric vehicles and solar products.
----------------------------------------------------------------------
🧠 Question 4: What is the speed of light?
📚 Retrieved Context: The speed of light is about 299,792 kilometers per second.
✅ Answer: The speed of light is about 299,792 kilometers per second.
----------------------------------------------------------------------
🧠 Question 5: Where does photosynthesis happen in plant cells?
📚 Retrieved Context: Photosynthesis occurs in the chloroplasts of plant cells.
✅ Answer: Chloroplasts.
----------------------------------------------------------------------